# Analysing Trained detectors

## Importing libraries

In [83]:
import gc
import os
import sys
import math
import glob
import tqdm
import random
import numpy as np
from tqdm import tqdm
import time 
from time import sleep

In [84]:
import pandas as pd
import xml.etree.ElementTree as ET

In [85]:
import cv2
import dlib
from imutils import face_utils
from skimage.feature import hog
from skimage import data,exposure

In [86]:
import matplotlib 
%matplotlib inline
import matplotlib.pyplot as plt

In [87]:
from ipywidgets import widgets, interactive, fixed

## Paths

In [88]:
path_openface_xml = '/media/amogh/Stuff/OpenFace_Outputs/Openface_outputs/oface_combined_csv/openface_404_folders_75_confidence.xml'

In [89]:
path_detector_1_xml = '/home/amogh/cmu/detect_faces/custom_hog_output_1523.xml'

In [90]:
path_detector_2_xml = '/home/amogh/cmu/detect_faces/custom_hog_output_2523.xml'

In [91]:
path_detector_3_xml = '/home/amogh/cmu/detect_faces/custom_hog_output_3523.xml'

## Computation functions

In [92]:
def box_percent_covered (a,b):
    inner = a.intersect(b).area()
    if inner == 0:
        return 0
    return max(inner/a.area(), inner/b.area())

In [93]:
def drect_add(a,b):
    """
    Being able to add two rectangles, as addition is not available in the version of dlib in anaconda environment.
    """
    
    return dlib.drectangle(min(a.left(),b.left()),min(a.top(),b.top()),max(a.right(),b.right()),max(a.bottom(),b.bottom()))

In [94]:
def intersect(a,b,iou_thresh=0.5,percent_covered_thresh=1.0):
    
    inner = a.intersect(b).area()
    if (inner == 0):
        return false
    
    outer = drect_add(a,b).area()
    print("inner,outer,a.area,b.area: ", inner, outer, a.area(),b.area())
    if (inner/outer > iou_thresh or 
        inner/a.area() > percent_covered_thresh or 
        inner/b.area() > percent_covered_thresh):
        return True
    else:
        return False

In [95]:
a = dlib.drectangle(2,1,4.5,3)
b = dlib.drectangle(3,1,5,3)

In [96]:
type(max(1.,2.))

float

In [97]:
a.left()

2.0

In [98]:
intersect(a,b)

('inner,outer,a.area,b.area: ', 7.5, 12.0, 10.5, 9.0)


True

## Reading xml files

There are 2 xml files. One with the ground truth and the other with the detected. Given the name of the image, need to get the boxes list from one, and the box list from other xml.

In [99]:
def getListImagesFromXML(path_xml):
    """
    Gets the list of the image filenames from an XML
    """
    
    tree = ET.parse(path_xml)
    root = tree.getroot()
    
    list_filenames = []
    for image in root.iter('image'):
        list_filenames.append(image.attrib['file'])
        
    return list_filenames

In [100]:
def getDetectedBoxes(image_file_name, path_detector_xml_file):
    """
    Get list of boxes detected
    """
    
    tree = ET.parse(path_openface_xml)
    root = tree.getroot()
    for image in root.iter('image'):
        #image found
        if image.attrib['file'] == image_file_name:
            if (len(image) == 0):
                return []
            else:
                list_boxes = []
                for box_num, box in enumerate(image):
                    box_attribs = box.attrib
                    box_rect = dlib.drectangle((float)(box_attribs['left']),
                                               (float)(box_attribs['top']),
                                              ((float)(box_attribs['left'])+(float)(box_attribs['width'])),
                                              ((float)(box_attribs['top'])+(float)(box_attribs['height'])))
                    list_boxes.append(box_rect)
                return list_boxes
    print('image not found')
    return None

In [104]:
def getHitStats(list_images, path_truth_xml_file, path_detector_xml_file):
    """
    Get the statistics such as total_hits, correct_hits and total_true targets from a list of image names,
    and the path to xml files.
    """
    
    correct_hits = 0
    total_true_targets = 0
    missing_detections = 0
    total_hits = 0

    for image in tqdm(list_images):
        
        detected_boxes = getDetectedBoxes(image, path_detector_xml_file)
        true_boxes = getDetectedBoxes(image, path_truth_xml_file)
        
        # If there are no boxes detected, add the true number of boxes to missing detections
        if (len(detected_boxes) == 0):
            missing_detections += len(true_boxes)
        
        # Go over all the labels (true boxes) and match them with all the detections.
        count = 0
        used = [False] * len(detected_boxes)
        for i in range(len(true_boxes)):
            
            #Compare a truth label with all the detected boxes, given that they are not yet detected/used
            found_match = False
            for j in range(len(detected_boxes)):
                
                if(used[j]):
                    continue
                
                if(intersect(true_boxes[i],detected_boxes[j])):
                    used[j] == True
                    found_match = True
                    count += 1
                    break
        
            if(not found_match):
                missing_detections += 1
        
        correct_hits += count
        total_true_targets += len(true_boxes)
        total_hits += used.count(True)

    return total_hits, correct_hits, total_true_targets

    

In [102]:
def getAnalysis(total_hits, correct_hits, total_true_targets):
    """
    Get final precision and recall
    """
    
    if (total_hits == 0):
        precision = 1
    else:
        precision = correct_hits / total_hits
        
    if (total_true_targets == 0):
        recall = 1
    else:
        recall = correct_hits / total_true_targets

In [ ]:
getHitStats()

In [ ]:
getAnalysis

In [ ]:
def main():
    list_images = getListImagesFromXML(path_openface_xml)
    getHitStats(list_images, path_openface_xml,path_detector_1_xml)
main()

  0%|          | 13/57234 [00:27<33:17:02,  2.09s/it]

('inner,outer,a.area,b.area: ', 198891.0, 198891.0, 198891.0, 198891.0)


  0%|          | 14/57234 [00:29<33:24:15,  2.10s/it]

('inner,outer,a.area,b.area: ', 197055.0, 197055.0, 197055.0, 197055.0)


  0%|          | 15/57234 [00:31<33:25:51,  2.10s/it]

...

  0%|          | 22/57234 [00:46<33:23:42,  2.10s/it]

('inner,outer,a.area,b.area: ', 215418.0, 215418.0, 215418.0, 215418.0)


  0%|          | 23/57234 [00:48<33:24:22,  2.10s/it]

('inner,outer,a.area,b.area: ', 213036.0, 213036.0, 213036.0, 213036.0)


  0%|          | 24/57234 [00:50<33:27:03,  2.10s/it]